In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = r'C:\Users\TH\Desktop\NN\cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def load_data(filename):#从文件里读取数据
    with open(filename,'rb') as f:
        data =  pickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
    
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
            
        self._data = np.vstack(all_data)
        self._data = self._data /127.5 -1 #归一化，很重要，数字太大实现不了
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data
            
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)#混排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):#返回batch_size个样本
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

    
    
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

#batch_data,batch_labels = train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)  #测试CifarData类可以正常工作




            

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']
(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [2]:
x = tf.placeholder(tf.float32,[None,3072])
y = tf.placeholder(tf.int64,[None])    

''''w = tf.get_variable('w',[x.get_shape()[-1],10],initializer=tf.random_normal_initializer(0,1))#3072*10
b = tf.get_variable('b',[10],initializer=tf.constant_initializer(0.0))#10

y_ = tf.matmul(x,w)+b#[None,3072]*[3072,10] = [None,10]'''
hidden1 = tf.layers.dense(x,100,activation=tf.nn.relu)#tf.layers.dense等价上面被注释的代码
hidden2 = tf.layers.dense(hidden1,100,activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2,50,activation=tf.nn.relu)
y_ = tf.layers.dense(hidden3,10)


#交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
#这个函数可以计算softmax 计算onehot 计算ylogy_

predict = tf.argmax(y_,1)
correct_prediction = tf.equal(predict,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    init.run()
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,accu_val,_ = sess.run([loss,accuracy,train_op],feed_dict={x:batch_data,y:batch_labels})
        
        if i%500 == 0 :
            print('[Train] step',i,' loss=',loss_val,' acc=',accu_val)
            
        if i%5000==0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('Test step',i+1,' acc=',test_acc)

[Train] step 0  loss= 2.458094  acc= 0.0
(10000, 3072)
(10000,)
Test step 1  acc= 0.10399999999999998
[Train] step 500  loss= 1.4137268  acc= 0.55
[Train] step 1000  loss= 1.9870561  acc= 0.3
[Train] step 1500  loss= 1.5780201  acc= 0.4
[Train] step 2000  loss= 1.5352547  acc= 0.45
[Train] step 2500  loss= 1.435537  acc= 0.6
[Train] step 3000  loss= 2.045877  acc= 0.45
[Train] step 3500  loss= 1.8860451  acc= 0.25
[Train] step 4000  loss= 1.6105436  acc= 0.5
[Train] step 4500  loss= 1.7093017  acc= 0.4
[Train] step 5000  loss= 1.0040276  acc= 0.7
(10000, 3072)
(10000,)
Test step 5001  acc= 0.47650000000000003
[Train] step 5500  loss= 1.342346  acc= 0.55


KeyboardInterrupt: 